_Autor:_    __Jesús Casado__ <br> _Revisión:_ __02/07/2020__ <br>


__Descripción__:<br>
Funciones para extraer, combinar y agregar las series de datos originales del SAIH Cantábrico.

__Cosas a corregir__ <br>



In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta

In [2]:
def corregir_errores(serie, n):
    """Convierte en NaN las partes de la serie en las que un mismo valor se repite al menos 'n' veces
    
    Parámetros:
    -----------
    serie:     series (n,). 
    n:         número de veces que se permite que se repita un mismo valor
    
    Salida:
    -------
    b:         series (m,). Serie corregida"""
    
    a = (serie.diff(1) == 0) & (serie != 0)
    b = serie.copy()

    i = 0
    while i < a.shape[0]:
        if a.iloc[i]:
            c, j = 0, i + 1
            while (a.iloc[j]) & (i < a.shape[0]):
                print('i = {0:>7}\tj = {1:>7}\ttotal = {2:>7}'.format(i, j, a.shape), end='\r')
                c += 1
                j += 1
            if c >= n:
                b.iloc[i:j-1] = np.nan
                i = j + 1
            else:
                i += 1
        else:
            i += 1
            
    return b

In [2]:
def corregir_valores(data):
    """Corrige valores superiores/inferiores a los máximos/mínimos asumibles para cada variable. P.ej. lluvia negativa.
    
    Parámetros:
    -----------
    data:      data frame (n, m). Serie de datos bruta
    
    Salida:
    -------
    data:      data frame (n, m). Serie de datos corregida
    """
    
    errormin = {'precipitacion_mm': 0, 'caudal_m³/s': 0, 'nivel_m': 0, 'temperatura_C': -10,
                'amonio_mg/l': 0, 'conductividad_microS/cm': 0, 'oxigeno_mg/l': 0, 'pH': 0,
                'temperaturaAgua_C': -10, 'turbidez_NTU': 0}# 'piezometro_m': 0, 'limnimetro_m': 0, 
    errormax = {'precipitacion_mm': 100, 'caudal_m³/s': 2000, 'nivel_m': 20, 'temperatura_C': 40,
                'amonio_mg/l': 1e4, 'conductividad_microS/cm': 1e6, 'oxigeno_mg/l': 1e3, 'pH': 15,
                'temperaturaAgua_C': 50, 'turbidez_NTU': 1e2}# 'piezometro_m': 1e3, 'limnimetro_m': 1e3,
    
    # corregir valores erróneos
    for col in data.columns:
        mask = (data[col] < errormin[col]) | (data[col] > errormax[col])
        data.loc[mask, col] = np.nan
    
    # corregir caudal nulo con nivel positivo
    if 'caudal_m³/s' in data.columns:
        mask = (data['nivel_m'] > 0) & (data['caudal_m³/s'] == 0)
        data.loc[mask, 'caudal_m³/s'] = np.nan
        
    return data

In [1]:
def corregir_repes(data, signames):
    """Encuentra variables (columnas) repetidas en los datos y los unifica en una única serie
    
    Parámetros:
    -----------
    data:      data frame (n, m). Serie de datos bruta
    signames:  dict. Para cada nombre de señal indica el nombre y unidad de la variable medida
    
    Salida:
    -------
    data:      data frame (n, o). Serie de datos corregida
    """
    
    # variables disponibles
    variables = [signames[col[5:]] for col in data.columns]
    # encontrar variables repetidas
    var = list(set(variables))
    varName = [v for v in var if np.sum([v == variable for variable in variables]) > 1]
    # posición y nombre de las columnas a corregir
    for name in varName:
        colPos = [i for i, v in enumerate(variables) if v == name]
        colName = data.columns[colPos]
        # definir la columna fuente y destino
        colDst = data.loc[:, colName].notnull().sum().idxmax()
        colSrc = [col for col in colName if col != colDst]
        # traspasar datos entre columnas y eliminar columna fuente
        if len(colSrc) > 1:
            temp = data[colSrc].mean(axis=1)
        else:
            temp = data[colSrc]
        mask = temp.notnull()
        data.loc[mask, colDst] = temp.loc[mask]
        data.drop(colSrc, axis=1, inplace=True)
        del colPos, colName, colSrc, colDst, temp
    
    return data

def corregir_repes(data, signames):
    """Encuentra variables (columnas) repetidas en los datos y los unifica en una única serie
    
    Parámetros:
    -----------
    data:      data frame (n, m). Serie de datos bruta
    signames:  dict. Para cada nombre de señal indica el nombre y unidad de la variable medida
    
    Salida:
    -------
    data:      data frame (n, o). Serie de datos corregida
    """
    
    # variables disponibles
    variables = [signames[col[5:]] for col in data.columns]
    # encontrar variables repetidas
    var = list(set(variables))
    varName = [v for v in var if np.sum([v == variable for variable in variables]) > 1]
    # posición y nombre de las columnas a corregir
    for name in varName:
        colPos = [i for i, v in enumerate(variables) if v == name]
        colName = data.columns[colPos]
        # definir la columna destino y fuente
        colDst = data.loc[:, colName].notnull().sum().idxmax()
        colSrc = [col for col in colName if col != colDst]
        # media de las señales
        data[colDst] = data[colName].mean(axis=1, skipna=True)
        # eliminar señales repetidas
        data.drop(colSrc, axis=1, inplace=True)
        del colPos, colName, colSrc, colDst
    
    return data

In [1]:
def SAIH_CHC(estacion, rutaorig, freq=None, rutaexp=None, verbose=True):
    """Genera las series diarias para las estaciones del SAIH Cantábrico.
    
    Entradas:
    ---------
    estacion:    str o int. Nombre de la estación
    ruta:        str.
    freq:        str. Resolución temporal a la que remuestrear los datos. Por defecto es 'None', es decir, se genera una serie con la resolución original cincominutal
    verbose:     boolean.
    
    Salidas:
    --------
    Genera un archivo .csv con la serie de la estación indicada con la resolución temporal indicada (o 5 min si 'freq' es None).
    Los archivos se guardan en una subcarpeta dentro de 'ruta' de nombre igual a la frecuencia de la serie.
    """

    # rutas
    rutaSAIH1 = rutaorig + '/Hasta junio de 2015/' + str(estacion) + '/'
    rutaSAIH2 = rutaorig + '/Desde julio de 2015/' + str(estacion) + '/'

    signames = {'AIPCINC': 'precipitacion_mm',
                'ACQRIO1': 'caudal_m³/s',
                'AINRIO1': 'nivel_m',
                'AINRIO2': 'nivel_m',
                'AINRL7S': 'nivel_m',
                'AITEMEX': 'temperatura_C',
                #'_PIEZO': 'piezometro_m',
                #'_POZO': 'piezometro_m',
                #'_LIMNI': 'limnimetro_m',
                'AIA3ATS': 'amonio_mg/l',
                'AIMPCTS': 'conductividad_microS/cm',
                'AIMPO2S': 'oxigeno_mg/l',
                'AIMPPHS': 'pH',
                'AIMPTTS': 'temperaturaAgua_C',
                'AITUTUS': 'turbidez_NTU'}
    redondeo = {'precipitacion_mm': 1, 'caudal_m³/s': 2, 'nivel_m': 2, 'temperatura_C': 1,
                'amonio_mg/l': 2, 'conductividad_microS/cm': 0, 'oxigeno_mg/l': 1, 'pH': 1,
                'temperaturaAgua_C': 1, 'turbidez_NTU': 0}# 'piezometro_m': 3, 'limnimetro_m': 3, 
    
    # PARTE 1
    # -------
    if os.path.exists(rutaSAIH1):
        # encontrar archivos de la estación
        files = [file for file in os.listdir(rutaSAIH1) if file[:4] == str(estacion)]

        # Importar datos cincominutales
        data1 = pd.DataFrame()
        for i, file in enumerate(files):
            print('{0:30}'.format(file), end='\r')
            # importar serie original
            aux = pd.read_csv(rutaSAIH1 + file, sep=';', encoding='latin-1', decimal=',',
                              low_memory=False, na_values=[-100, 65535, 6523.6, -816])
            aux.dropna(axis=0, how='all', inplace=True)
            aux.Fecha = [datetime.strptime(date, '%d/%m/%Y %H:%M') for date in aux.Fecha]
            aux.set_index('Fecha', drop=True, inplace=True)
            aux.loc[aux.Calidad == 3, 'Valor'] = np.nan # eliminar datos con baja calidad
            aux.loc[aux.Calidad == 5, 'Valor'] = np.nan # eliminar datos con baja calidad
            aux.loc[aux.Calidad == -6, 'Valor'] = np.nan # eliminar datos con baja calidad
            # reordenar 'aux' por señales
            signals = [signal for signal in aux['Nombre señal'].unique() if signal[-7:] in list(signames.keys())]
            aux2 = pd.DataFrame(index=pd.date_range(aux.index.min(), aux.index.max(), freq='5min'))#columns=cols)
            for signal in signals:
                new_col = '{0}X{1}'.format(estacion, signal[-7:])
                temp = aux.loc[aux['Nombre señal'] == signal, 'Valor']
                aux2.loc[temp.index, new_col] = temp
#             aux2 = pd.DataFrame()#columns=cols)
#             for signal in signals:
#                 new_col = '{0}X{1}'.format(estacion, signal[-7:])
#                 temp = aux.loc[aux['Nombre señal'] == signal, 'Valor']
#                 try:
#                     cols = list(aux2.columns)
#                     aux2 = pd.concat((aux2, temp), axis=1, sort=True)
#                     aux2.columns = cols + [new_col]
#                 except:
#                     At = temp.index[1] - temp.index[0]
#                     At = int(At.seconds / 60) # minutos
#                     if At != 5:
#                         temp2 = pd.Series(index=pd.date_range(aux2.index[0], aux2.index[-1], freq='5min'),
#                                           dtype='float')
#                         temp2[temp.index] = temp
#                         aux2[new_col] = temp2
            # concatenar a la serie generada
            data1 = pd.concat((data1, aux2), axis=0, sort=True)
        del files

    # PARTE 2
    # -------
    if os.path.exists(rutaSAIH2):
        # encontrar archivos de la estación
        files = os.listdir(rutaSAIH2)
        # corregir nombre del archivo si fuera necesario
        for i, file in enumerate(files):
            if len(file) > 16:
                new_file = file[:12] + file[-4:]
                try:
                    os.rename(ruta_stn + file, ruta_stn + new_file)
                    files[i] = new_file
                except:
                    continue
        # Importar datos cincominutales
        data2 = pd.DataFrame()
        for file in files:
            print('{0:30}'.format(file), end='\r')
            aux = pd.read_csv(rutaSAIH2 + file, sep=';', decimal=',', encoding='latin-1', skiprows=1)
            aux['Fecha/Hora'] = [datetime.strptime(date, '%d/%m/%Y %H:%M') for date in aux['Fecha/Hora']]
            aux.set_index('Fecha/Hora', drop=True, inplace=True)
            aux.index.name = 'Fecha'
            cols = [col for col in aux.columns if col[5:] in list(signames.keys())]
            aux = aux.loc[:, cols]
            data2 = pd.concat((data2, aux), axis=0, sort=True)
        del files
    
    # UNIR SERIES
    # -----------
    # unir las dos series como serie minutal para evitar errores
    if ('data1' in locals()) and ('data2' in locals()):
        data = pd.concat((data1, data2), axis=0, sort=True)
    elif ('data1' in locals()) and ('data2' not in locals()):
        data = data1
    elif ('data2' in locals()) and ('data1' not in locals()): 
        data = data2

    # combinar columnas con la misma variable y cambiar nombre de las columnas
    data = corregir_repes(data, signames)
    # corregir nombre de las columnas de señales a variables
    data.index.name = 'Fecha'
    data.columns = [signames[col[5:]] for col in data.columns if col[5:] in list(signames.keys())]
    # corregir valores
    data = corregir_valores(data)
    
    # AGREGAR DATOS A LA FRECUENCIA DESEADA
    # -------------------------------------
    data_ag = data.resample(freq).mean()
    if 'precipitacion_mm' in data.columns:
        data_ag['precipitacion_mm'] *= data['precipitacion_mm'].resample(freq).count()
    data_ag = data_ag.astype(float).round(redondeo)
    # recortar serie al primer y último dato útil
    st, en = data_ag.first_valid_index(), data_ag.last_valid_index()
    data_ag = data_ag.loc[st:en, :]
    # ordenar columnas alfabéticamente
    data_ag.sort_index(axis=1, inplace=True)
        
    # EXPORTAR
    # --------
    # exportar la serie remuestreada
    if (freq != None) and (rutaexp != None):
        ruta_ag = rutaexp + freq + '/'
        if not os.path.exists(ruta_ag):
            os.makedirs(ruta_ag)
        # generar encabezado
        header = ['inicio\t{0}\n'.format(data_ag.index[0])]
        header.append('At\t\t{0}\n'.format(freq))
        header.append('n\t\t{0}\n\n'.format(data_ag.shape[0]))
        # escribir archivo nuevo
        with open(ruta_ag + str(estacion) + '.csv', 'w') as f:
            for line in header:
                f.write(line)
            data_ag.to_csv(f, index=False, line_terminator='\n', sep=',', na_rep='', encoding='latin1')
#         data_ag.to_csv(ruta_ag + str(estacion) + '.csv', sep=',', na_rep='', encoding='latin1')#,
#                        #float_format='%.3f')

    if verbose == True:
        print('nº de días de la serie:\t', data_ag.shape[0])
        print('variables:\t', list(data_ag.columns))
        
    SAIH_CHC.data = data_ag